# Shelter Animal Outcomes 5

## Naive Bayes

In [1]:
from sklearn.naive_bayes import GaussianNB 
from sklearn import cross_validation
import pandas as pd

In [2]:
df_train = pd.read_csv('../Shelter_train.csv')
df_test = pd.read_csv('../Shelter_test.csv')

In [3]:
x = df_train.ix[:, :-1]
y = df_train.ix[:, -1]
df_test = df_test.drop('ID', 1)

In [4]:
clf = GaussianNB()
cross_validation.cross_val_score(clf, x, y, scoring="log_loss")

array([-1.22157213, -1.26583667, -1.20188548])

In [5]:
clf = clf.fit(x, y)
predictions = clf.predict_proba(df_test)
output = pd.DataFrame(predictions, columns=['Adoption', 'Died', 'Euthanasia', 'Return_to_owner', 'Transfer'])
output.index.names = ['ID']
output.index += 1
output.head()

,Adoption,Died,Euthanasia,Return_to_owner,Transfer
1,0.003758,0.007522,0.064255,0.207911,0.716553
2,0.469521,0.000370,0.007526,0.459204,0.063379
3,0.845995,0.001081,0.007876,0.013983,0.131064
4,0.154246,0.002755,0.029189,0.515097,0.298713
5,0.523129,0.000114,0.004230,0.436010,0.036515


In [6]:
output.to_csv('../submission-GaussianNB.1.0.csv', index_label = 'ID')